# Relationship Management with Nepal Entity Service

This notebook demonstrates how to manage relationships between entities using the Nepal Entity Service (nes2). We'll work with authentic Nepali political relationships including party memberships, government positions, and organizational affiliations.

## Topics Covered

1. Create relationships between entities
2. Add temporal information (start/end dates)
3. Query relationships by entity
4. Query relationships by type
5. Update relationships
6. View relationship version history
7. Explore relationship networks

## 1. Setup and Initialization

In [ ]:
# Import required modules
from pathlib import Path
from datetime import date

from nes2.database.file_database import FileDatabase
from nes2.services.publication import PublicationService
from nes2.services.search import SearchService

In [ ]:
# Initialize database and services
db_path = Path("../nes-db/v2")
db = FileDatabase(base_path=str(db_path))

pub_service = PublicationService(database=db)
search_service = SearchService(database=db)

print("✓ Services initialized successfully")

## 2. Create Party Membership Relationships

Let's create MEMBER_OF relationships between politicians and their political parties.

In [ ]:
# Define party membership relationships
memberships = [
    {
        "politician": "entity:person/pushpa-kamal-dahal",
        "party": "entity:organization/political_party/nepal-communist-party-maoist-centre",
        "role": "Chairman",
        "start_date": date(1994, 1, 1),
        "founding_member": True,
    },
    {
        "politician": "entity:person/sher-bahadur-deuba",
        "party": "entity:organization/political_party/nepali-congress",
        "role": "President",
        "start_date": date(1971, 1, 1),
        "senior_leader": True,
    },
    {
        "politician": "entity:person/khadga-prasad-oli",
        "party": "entity:organization/political_party/cpn-uml",
        "role": "Chairman",
        "start_date": date(1991, 1, 1),
        "senior_leader": True,
    },
]

print(f"Preparing to create {len(memberships)} party membership relationships")

In [ ]:
# Create the relationships
created_relationships = []

for membership in memberships:
    # Verify entities exist
    politician = await pub_service.get_entity(membership["politician"])
    party = await pub_service.get_entity(membership["party"])

    if not politician:
        print(f"⚠ Politician not found: {membership['politician']}")
        continue

    if not party:
        print(f"⚠ Party not found: {membership['party']}")
        continue

    # Create relationship
    try:
        relationship = await pub_service.create_relationship(
            source_entity_id=membership["politician"],
            target_entity_id=membership["party"],
            relationship_type="MEMBER_OF",
            start_date=membership["start_date"],
            attributes={
                "role": membership["role"],
                "founding_member": membership.get("founding_member", False),
                "senior_leader": membership.get("senior_leader", False),
            },
            author_id="author:human:notebook-user",
            change_description=f"{politician.names[0].en.full} membership in {party.names[0].en.full}",
        )

        created_relationships.append(relationship)
        print(f"✓ Created: {politician.names[0].en.full} → {party.names[0].en.full}")
        print(f"  Role: {membership['role']}")
        print(f"  Since: {membership['start_date']}")
        print(f"  ID: {relationship.id}\n")

    except Exception as e:
        print(f"❌ Failed: {e}\n")

print(f"\nTotal relationships created: {len(created_relationships)}")

## 3. Query Relationships by Entity

Let's find all relationships for a specific politician.

In [ ]:
# Query relationships for Pushpa Kamal Dahal
entity_id = "entity:person/pushpa-kamal-dahal"

# Get the entity first
entity = await pub_service.get_entity(entity_id)
if entity:
    print(f"Relationships for: {entity.names[0].en.full} ({entity.names[0].ne.full})")
    print("=" * 70)

# Find all relationships where this person is the source
relationships = await search_service.search_relationships(
    source_entity_id=entity_id, limit=10
)

if relationships:
    print(f"\nFound {len(relationships)} relationship(s):\n")

    for rel in relationships:
        target = await pub_service.get_entity(rel.target_entity_id)
        if target:
            print(f"Type: {rel.type}")
            print(f"Target: {target.names[0].en.full}")
            print(f"Since: {rel.start_date or 'Unknown'}")
            if rel.attributes:
                print(f"Attributes:")
                for key, value in rel.attributes.items():
                    print(f"  - {key}: {value}")
            print()
else:
    print("No relationships found")

## 4. Query Relationships by Type

Let's find all MEMBER_OF relationships in the database.

In [ ]:
# Query all MEMBER_OF relationships
all_memberships = await search_service.search_relationships(
    relationship_type="MEMBER_OF", limit=20
)

print(f"All MEMBER_OF Relationships")
print("=" * 70)
print(f"Found {len(all_memberships)} membership(s)\n")

for rel in all_memberships:
    source = await pub_service.get_entity(rel.source_entity_id)
    target = await pub_service.get_entity(rel.target_entity_id)

    if source and target:
        print(f"{source.names[0].en.full}")
        print(f"  → Member of: {target.names[0].en.full}")
        if rel.attributes and "role" in rel.attributes:
            print(f"  → Role: {rel.attributes['role']}")
        if rel.start_date:
            print(f"  → Since: {rel.start_date}")
        print()

## 5. Create Government Position Relationships

Let's create relationships between politicians and government positions.

In [ ]:
# Define government position relationships
positions = [
    {
        "politician": "entity:person/ram-chandra-poudel",
        "position": "entity:organization/government_body/office-of-president",
        "role": "President of Nepal",
        "start_date": date(2023, 3, 13),
        "term_length": "5 years",
    },
    {
        "politician": "entity:person/pushpa-kamal-dahal",
        "position": "entity:organization/government_body/office-of-prime-minister",
        "role": "Prime Minister",
        "start_date": date(2022, 12, 26),
        "term_number": 3,
    },
]

print(f"Creating {len(positions)} government position relationships\n")

for position in positions:
    politician = await pub_service.get_entity(position["politician"])
    gov_body = await pub_service.get_entity(position["position"])

    if not politician or not gov_body:
        print(f"⚠ Skipping: Entity not found")
        continue

    try:
        relationship = await pub_service.create_relationship(
            source_entity_id=position["politician"],
            target_entity_id=position["position"],
            relationship_type="HOLDS_POSITION",
            start_date=position["start_date"],
            attributes={
                "role": position["role"],
                "term_length": position.get("term_length"),
                "term_number": position.get("term_number"),
            },
            author_id="author:human:notebook-user",
            change_description=f"{politician.names[0].en.full} appointed as {position['role']}",
        )

        print(f"✓ Created: {politician.names[0].en.full}")
        print(f"  Position: {position['role']}")
        print(f"  Since: {position['start_date']}")
        print(f"  ID: {relationship.id}\n")

    except Exception as e:
        print(f"❌ Failed: {e}\n")

## 6. Update a Relationship

Let's update a relationship to add an end date (when someone leaves a position).

In [ ]:
# Find a relationship to update
relationships = await search_service.search_relationships(
    source_entity_id="entity:person/pushpa-kamal-dahal",
    relationship_type="HOLDS_POSITION",
    limit=1,
)

if relationships:
    rel = relationships[0]
    print(f"Current relationship:")
    print(f"  ID: {rel.id}")
    print(f"  Type: {rel.type}")
    print(f"  Start: {rel.start_date}")
    print(f"  End: {rel.end_date or 'Current'}")
    print(f"  Version: {rel.version_summary.version_number}")

    # Update to add end date (hypothetical)
    rel.end_date = date(2024, 7, 15)
    rel.attributes["end_reason"] = "End of term"

    updated_rel = await pub_service.update_relationship(
        relationship=rel,
        author_id="author:human:notebook-user",
        change_description="Added end date for term completion",
    )

    print(f"\n✓ Relationship updated!")
    print(f"  New version: {updated_rel.version_summary.version_number}")
    print(f"  End date: {updated_rel.end_date}")
    print(f"  End reason: {updated_rel.attributes.get('end_reason')}")
else:
    print("No relationship found to update")

## 7. View Relationship Version History

In [ ]:
# Get version history for a relationship
if created_relationships:
    rel = created_relationships[0]

    print(f"Version History for Relationship: {rel.id}")
    print("=" * 70)

    versions = await pub_service.get_relationship_versions(rel.id)
    print(f"Total versions: {len(versions)}\n")

    for version in versions:
        print(f"Version {version.version_number}:")
        print(f"  Created: {version.created_at}")
        print(f"  Author: {version.author.slug}")
        print(f"  Description: {version.change_description or '(no description)'}")

        if version.snapshot:
            snapshot = version.snapshot
            print(f"  Type: {snapshot.get('type')}")
            print(f"  Start: {snapshot.get('start_date')}")
            print(f"  End: {snapshot.get('end_date') or 'Current'}")
            if "attributes" in snapshot:
                print(f"  Attributes: {list(snapshot['attributes'].keys())}")
        print()
else:
    print("No relationships available for version history")

## 8. Explore Relationship Networks

Let's visualize the relationship network for a politician.

In [ ]:
# Build a relationship network for a politician
entity_id = "entity:person/pushpa-kamal-dahal"
entity = await pub_service.get_entity(entity_id)

if entity:
    print(f"Relationship Network for: {entity.names[0].en.full}")
    print("=" * 70)

    # Get all relationships
    relationships = await search_service.search_relationships(
        source_entity_id=entity_id, limit=20
    )

    # Group by relationship type
    by_type = {}
    for rel in relationships:
        rel_type = rel.type
        if rel_type not in by_type:
            by_type[rel_type] = []
        by_type[rel_type].append(rel)

    # Display grouped relationships
    for rel_type, rels in by_type.items():
        print(f"\n{rel_type} ({len(rels)}):")
        print("-" * 70)

        for rel in rels:
            target = await pub_service.get_entity(rel.target_entity_id)
            if target:
                print(f"  → {target.names[0].en.full}")
                if rel.attributes and "role" in rel.attributes:
                    print(f"     Role: {rel.attributes['role']}")
                if rel.start_date:
                    status = "Current" if not rel.end_date else f"Until {rel.end_date}"
                    print(f"     Period: {rel.start_date} - {status}")
else:
    print("Entity not found")

## 9. Query Relationships by Target Entity

Let's find all politicians who are members of a specific party.

In [ ]:
# Find all members of Nepali Congress
party_id = "entity:organization/political_party/nepali-congress"
party = await pub_service.get_entity(party_id)

if party:
    print(f"Members of: {party.names[0].en.full} ({party.names[0].ne.full})")
    print("=" * 70)

    # Find all MEMBER_OF relationships targeting this party
    members = await search_service.search_relationships(
        target_entity_id=party_id, relationship_type="MEMBER_OF", limit=20
    )

    print(f"\nFound {len(members)} member(s):\n")

    for rel in members:
        member = await pub_service.get_entity(rel.source_entity_id)
        if member:
            print(f"{member.names[0].en.full}")
            if rel.attributes and "role" in rel.attributes:
                print(f"  Role: {rel.attributes['role']}")
            if rel.start_date:
                print(f"  Member since: {rel.start_date}")
            print()
else:
    print("Party not found")

## 10. Summary and Next Steps

In this notebook, we've learned how to:

- ✓ Create relationships between entities
- ✓ Add temporal information (start/end dates)
- ✓ Query relationships by source entity
- ✓ Query relationships by type
- ✓ Query relationships by target entity
- ✓ Update relationships with versioning
- ✓ View relationship version history
- ✓ Explore relationship networks

### Next Steps

1. Learn about **Data Import Workflows** in `03_data_import_workflow.ipynb`
2. Perform **Data Quality Analysis** in `04_data_quality_analysis.ipynb`

### Resources

- Data Maintainer Guide: `docs/data-maintainer-guide.md`
- Example Scripts: `examples/`
- API Documentation: Run the server and visit `/docs`